## **`Challenge 6 `**
    
## <center>Acerca de los datos </center>

​**Este conjunto de datos contiene 2126 registros de características extraídas de exámenes de Cardiotocograma, que luego fueron clasificados por un obstetra experto en 3 clases: "Normal", "Suspect" y "Pathological". El conjunto de datos contiene las siguientes características:**
    
* **baseline value:** Valor de línea base de la frecuencia cardíaca fetal (FHR) (latidos por minuto)   
* **accelerations:** Número de aceleraciones por segundo
* **fetal_movement:** Número de movimientos fetales por segundo    
* **uterine_contractions:** Número de contracciones uterinas por segundo
* **light_decelerations:** Número de desaceleraciones leves (LDs) por segundo
* **severe_decelerations:** Número de desaceleraciones severas (SDs) por segundo
* **prolongued_decelerations:** Número de desaceleraciones prolongadas (PDs) por segundo
* **abnormal_short_term_variability:** Porcentaje de tiempo con variabilidad anormal a corto plazo
* **mean_value_of_short_term_variability:** Valor medio de la variabilidad a corto plazo
* **percentage_of_time_with_abnormal_long_term_variability:** Porcentaje de tiempo con variabilidad anormal a largo plazo
* **mean_value_of_long_term_variability:** Valor medio de la variabilidad a largo plazo
* **histogram_width:** Ancho del histograma creado usando todos los valores de un registro
* **histogram_min:** Valor mínimo del histograma
* **histogram_max:** Valor máximo del histograma
* **histogram_number_of_peaks:** Número de picos en el histograma del examen
* **histogram_number_of_zeroes:** Número de ceros en el histograma del examen
* **histogram_mode:** Modo del histograma
* **histogram_mean:** Media del histograma
* **histogram_median:** Mediana del histograma
* **histogram_variance:** Varianza del histograma
* **histogram_tendency:** Tendencia del histograma
* **fetal_health:** Codificado como 1-Normal(Normal); 2-Sospechoso(Suspect); 3-Patológic(Pathological).



<br>

Datos extraidos de :

https://www.kaggle.com/datasets/andrewmvd/fetal-health-classification <br>

<br>

In [30]:
import pandas as pd

# Cargar los datos
data_path = "https://github.com/ulewis/Ejemplos/raw/main/Datos/fetal_health.csv"
data = pd.read_csv(data_path)

# Verificar si hay valores faltantes
missing_values = data.isnull().sum()
print(missing_values)


baseline value                                            0
accelerations                                             0
fetal_movement                                            0
uterine_contractions                                      0
light_decelerations                                       0
severe_decelerations                                      0
prolongued_decelerations                                  0
abnormal_short_term_variability                           0
mean_value_of_short_term_variability                      0
percentage_of_time_with_abnormal_long_term_variability    0
mean_value_of_long_term_variability                       0
histogram_width                                           0
histogram_min                                             0
histogram_max                                             0
histogram_number_of_peaks                                 0
histogram_number_of_zeroes                                0
histogram_mode                          

In [31]:
import logging
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Activar el registro
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# Dividir los datos en conjuntos de entrenamiento y prueba
X = data.drop('fetal_health', axis=1)
y = data['fetal_health']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un pipeline que incluye estandarización y SVC
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(random_state=42))
])

# Configuración para Grid Search
param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__gamma': ['scale', 'auto'],
    'svc__kernel': ['linear', 'rbf']
}

# Realizar Grid Search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=10)
grid_search.fit(X_train, y_train)

# Mejores parámetros y mejor puntuación
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de CV:", grid_search.best_score_)

# Evaluar el modelo en el conjunto de prueba
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START svc__C=0.1, svc__gamma=scale, svc__kernel=linear...........
[CV 1/5; 1/12] END svc__C=0.1, svc__gamma=scale, svc__kernel=linear;, score=0.932 total time=   0.0s
[CV 2/5; 1/12] START svc__C=0.1, svc__gamma=scale, svc__kernel=linear...........
[CV 2/5; 1/12] END svc__C=0.1, svc__gamma=scale, svc__kernel=linear;, score=0.885 total time=   0.0s
[CV 3/5; 1/12] START svc__C=0.1, svc__gamma=scale, svc__kernel=linear...........
[CV 3/5; 1/12] END svc__C=0.1, svc__gamma=scale, svc__kernel=linear;, score=0.912 total time=   0.0s
[CV 4/5; 1/12] START svc__C=0.1, svc__gamma=scale, svc__kernel=linear...........
[CV 4/5; 1/12] END svc__C=0.1, svc__gamma=scale, svc__kernel=linear;, score=0.879 total time=   0.0s
[CV 5/5; 1/12] START svc__C=0.1, svc__gamma=scale, svc__kernel=linear...........
[CV 5/5; 1/12] END svc__C=0.1, svc__gamma=scale, svc__kernel=linear;, score=0.906 total time=   0.0s
[CV 1/5; 2/12] START svc__C=0

In [33]:
import numpy as np

# Supongamos que tenemos un nuevo registro de características (21 caracteristicas)
new_data = np.array([[120, 0.003, 0, 0.006, 0.003, 0, 0, 27, 0.5, 32, 12, 64, 62, 130, 120, 0, 120, 137, 121, 73, 1]])

# Asegúrate de que las características estén en el mismo orden que en el conjunto de entrenamiento y tenga la forma correcta
new_data_scaled = grid_search.best_estimator_.named_steps['scaler'].transform(new_data)

# Predecir la salud fetal usando el pipeline
predicted_health = grid_search.predict(new_data_scaled)

# Definir que significa el numero de salud fetal
fetal_health = {1: "Normal", 2: "Suspect", 3: "Pathological"}

# Imprimir la clase predicha
print("La salud fetal predicha es:", fetal_health[int(predicted_health)])

La salud fetal predicha es: Pathological
